In [1]:
bucket_file = "s3://ms-syntheamass-1m-mldata/syntheamassCSV/CVX/web_cvx.csv"

cvx = spark.read.format("csv").option("inferSchema", True).option("header", True).load(bucket_file)

cvx.show()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1596204340474_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------------------+--------------------+--------------------+-------------+----------+----------+-----------+----+
|CVX Code|CVX Short Description|   Full Vaccine Name|                Note|VaccineStatus|internalID|nonvaccine|update_date| _c8|
+--------+---------------------+--------------------+--------------------+-------------+----------+----------+-----------+----+
|      54|   adenovirus, type 4|adenovirus vaccin...|                null|     Inactive|         1|     false|  28-May-10|null|
|      55|   adenovirus, type 7|adenovirus vaccin...|                null|     Inactive|         2|     false|  28-May-10|null|
|      82| adenovirus, unspe...|adenovirus vaccin...|This CVX code all...|     Inactive|         3|     false|  30-Sep-10|null|
|      24|              anthrax|     anthrax vaccine|                null|       Active|         4|     false|  11-Jun-19|null|
|      19|                  BCG|Bacillus Calmette...|                null|       Active|         5|     

In [2]:
bucket_file = "s3://ms-syntheamass-1m-mldata/syntheamassCSV/output_10/csv/immunizations.csv"

immunizations = spark.read.format("csv").option("inferSchema", True).option("header", True).load(bucket_file)

immunizations.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+--------------------+----+--------------------+
|               DATE|             PATIENT|           ENCOUNTER|CODE|         DESCRIPTION|
+-------------------+--------------------+--------------------+----+--------------------+
|2015-02-10 00:00:00|9bde2b07-b972-495...|0cc89db7-d544-49b...|   8|Hep B  adolescent...|
|2015-03-14 00:00:00|9bde2b07-b972-495...|1bd91587-d460-404...|   8|Hep B  adolescent...|
|2015-04-12 00:00:00|9bde2b07-b972-495...|07237304-bd9b-4f5...| 119|rotavirus  monova...|
|2015-04-12 00:00:00|9bde2b07-b972-495...|07237304-bd9b-4f5...|  20|                DTaP|
|2015-04-12 00:00:00|9bde2b07-b972-495...|07237304-bd9b-4f5...|  49|       Hib (PRP-OMP)|
|2015-04-12 00:00:00|9bde2b07-b972-495...|07237304-bd9b-4f5...| 133|Pneumococcal conj...|
|2015-04-12 00:00:00|9bde2b07-b972-495...|07237304-bd9b-4f5...|  10|                 IPV|
|2015-08-20 00:00:00|9bde2b07-b972-495...|073581e9-e17d-463...|   8|Hep B  adolescent...|
|2015-08-2

In [3]:
cvx.createOrReplaceTempView("cvx")
immunizations.createOrReplaceTempView("immunizations")

spark.sql("""
select c.`CVX Code`, c.`CVX Short Description`,
i.DESCRIPTION, i.CODE 
from cvx c 
join immunizations i
on c.`CVX Code` = i.CODE
group by 1, 2, 3, 4""").show(truncate = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------------------+--------------------+----+
|CVX Code|CVX Short Description|         DESCRIPTION|CODE|
+--------+---------------------+--------------------+----+
|     119| rotavirus, monova...|rotavirus  monova...| 119|
|      33| pneumococcal poly...|pneumococcal poly...|  33|
|     133| Pneumococcal conj...|Pneumococcal conj...| 133|
|       8| Hep B, adolescent...|Hep B  adolescent...|   8|
|      20|                 DTaP|                DTaP|  20|
|     114|  meningococcal MCV4P| meningococcal MCV4P| 114|
|     115|                 Tdap|                Tdap| 115|
|     140| Influenza, season...|Influenza  season...| 140|
|      10|                  IPV|                 IPV|  10|
|      21|            varicella|           varicella|  21|
|     113| Td (adult), 5 Lf ...|Td (adult) preser...| 113|
|      62|    HPV, quadrivalent|   HPV  quadrivalent|  62|
|       3|                  MMR|                 MMR|   3|
|      83| Hep A, ped/adol, ...|Hep A  ped/adol  ...|  8

In [25]:
cvx.createOrReplaceTempView("cvx")
immunizations.createOrReplaceTempView("immunizations")

spark.sql("""
select * 
from immunizations i
where i.CODE not in (select `CVX Code` from cvx)""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---------+----+-----------+
|DATE|PATIENT|ENCOUNTER|CODE|DESCRIPTION|
+----+-------+---------+----+-----------+
+----+-------+---------+----+-----------+

In [26]:
immunizations.createOrReplaceTempView("immunizations")

spark.sql(""" select * from immunizations where CODE is null """).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---------+----+-----------+
|DATE|PATIENT|ENCOUNTER|CODE|DESCRIPTION|
+----+-------+---------+----+-----------+
+----+-------+---------+----+-----------+

In [31]:
# Natural Language Processing to determin the distance between two strings
# This is defined as the number of insertions needed for the strings to match

def levenshtein(s1,s2): 
    if len(s1) > len(s2):
        s1,s2 = s2,s1 
    distances = range(len(s1) + 1) 
    for index2,char2 in enumerate(s2):
        newDistances = [index2+1]
        for index1,char1 in enumerate(s1):
            if char1 == char2:
                newDistances.append(distances[index1]) 
            else:
                 newDistances.append(1 + min((distances[index1], distances[index1+1], newDistances[-1]))) 
        distances = newDistances 
    return distances[-1]

print(levenshtein("Analyze", 'analyse'))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2

In [4]:
cvx.createOrReplaceTempView("cvx")
immunizations.createOrReplaceTempView("immunizations")

spark.sql("""
select c.`CVX Short Description`,
i.DESCRIPTION
from cvx c 
join immunizations i
on c.`CVX Code` = i.CODE
group by 1, 2""").show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------+--------------------------------------------------+
|CVX Short Description                                       |DESCRIPTION                                       |
+------------------------------------------------------------+--------------------------------------------------+
|Hep A, ped/adol, 2 dose                                     |Hep A  ped/adol  2 dose                           |
|Influenza, seasonal, injectable, preservative free          |Influenza  seasonal  injectable  preservative free|
|Hib (PRP-OMP)                                               |Hib (PRP-OMP)                                     |
|Hep B, adolescent or pediatric                              |Hep B  adolescent or pediatric                    |
|zoster live                                                 |zoster                                            |
|rotavirus, monovalent                                       |rotavirus  monovalent     

In [6]:
sc.install_pypi_package("fuzzywuzzy")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/43/ff/74f23998ad2f93b945c0309f825be92e04e0348e062026998b5eefef4c33/fuzzywuzzy-0.18.0-py2.py3-none-any.whl

In [7]:
# Determining the levenshtein ratio between strings
# this measures how closely the strings match 


from fuzzywuzzy import fuzz

Str1 = "Td (adult), 5 Lf tetanus toxoid, preservative free, adsorbed"
Str2 = "Td (adult) preservative free"

Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
Partial_Ratio = fuzz.partial_ratio(Str1.lower(),Str2.lower())
Token_Sort_Ratio = fuzz.token_sort_ratio(Str1, Str2)
Token_Set_Ratio = fuzz.token_set_ratio(Str1, Str2)

print(Ratio)
print(Partial_Ratio)
print(Token_Sort_Ratio)
print(Token_Set_Ratio) # uses preprocessing.
# instead of just tokenizing strings, storting and pasting together,
# token_set_ratio performs a set operation that takes out the common tokens 
# and then makes fuzz.ratio() pairwise comparisons btwn the following new strings


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

64
71
64
100
/tmp/1596216688852-0/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')

In [8]:
# apply token_set or sort or partial to the csv and create a new col with the scores
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

descriptions = spark.sql("""
select c.`CVX Short Description` as cvx_description,
i.DESCRIPTION
from cvx c 
join immunizations i
on c.`CVX Code` = i.CODE
group by 1, 2""")

token_set_ratio = udf(fuzz.token_set_ratio)

descriptions = descriptions.withColumn("levenshtein_ratio", 
                                       token_set_ratio(descriptions.cvx_description, descriptions.DESCRIPTION))

descriptions.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----------------+
|     cvx_description|         DESCRIPTION|levenshtein_ratio|
+--------------------+--------------------+-----------------+
|Hep A, ped/adol, ...|Hep A  ped/adol  ...|              100|
|Influenza, season...|Influenza  season...|              100|
|       Hib (PRP-OMP)|       Hib (PRP-OMP)|              100|
|Hep B, adolescent...|Hep B  adolescent...|              100|
|         zoster live|              zoster|              100|
|rotavirus, monova...|rotavirus  monova...|              100|
| meningococcal MCV4P| meningococcal MCV4P|              100|
|pneumococcal poly...|pneumococcal poly...|               90|
|                Tdap|                Tdap|              100|
|                 IPV|                 IPV|              100|
|                 MMR|                 MMR|              100|
|   HPV, quadrivalent|   HPV  quadrivalent|              100|
|Pneumococcal conj...|Pneumococcal conj...|              100|
|Td (adu